- 변수 이름을 loop 돌려서 할당하기

https://m.blog.naver.com/PostView.nhn?blogId=nomadgee&logNo=220857820094&proxyReferer=https%3A%2F%2Fwww.google.com%2F

In [1]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import math
from sklearn.metrics import mean_squared_error

import tensorflow as tf

In [2]:
os.getcwd()

'D:\\Mywork\\DACON_CONTEST_2019\\SUHYUN'

In [3]:
df = pd.read_csv('./결과/TEST_DHweightDaconArima_NAFilled.csv')
df['Time'] = pd.to_datetime(df['Time']) 
df = df.set_index('Time')

In [4]:
df

,x5,x7,x10,x16,x18,x20,x26,x30,x31,x32,...,x455,x457,x459,x460,x462,x468,x470,x472,x473,x481
Time,,,,,,,,,,,,,,,,,,,,,
2017-08-29 10:00:00,6.373730,1.359795,2.947804,3.791421,2.518855,1.148489,1.453803,3.498120,0.481148,0.496201,...,0.176,0.251,0.198,0.092,0.268794,0.225946,0.371684,0.294292,0.202067,0.254667
2017-08-29 11:00:00,6.373905,1.361594,2.966805,3.802763,2.519404,1.150483,1.458903,3.461189,0.485545,0.497116,...,0.176,0.239,0.209,0.093,0.268782,0.225983,0.371698,0.294284,0.202070,0.254625
2017-08-29 12:00:00,6.374295,1.363109,2.982248,3.811819,2.519835,1.151945,1.462694,3.430963,0.488986,0.497688,...,0.197,0.172,0.492,0.073,0.268777,0.226005,0.371707,0.294280,0.202072,0.254596
2017-08-29 13:00:00,6.374804,1.364386,2.994801,3.819050,2.520170,1.153084,1.465513,3.406225,0.491679,0.498046,...,0.170,0.142,0.149,0.090,0.268776,0.226019,0.371713,0.294277,0.202075,0.254576
2017-08-29 14:00:00,6.375378,1.365461,3.005003,3.824822,2.520428,1.154028,1.467609,3.385979,0.493787,0.498269,...,0.198,0.141,0.130,0.082,0.268778,0.226027,0.371717,0.294275,0.202077,0.254562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-06-30 19:00:00,6.937000,1.479000,2.685000,7.788000,2.490000,11.546000,1.255000,10.449000,0.449000,0.744000,...,0.277,0.743,0.091,0.584,0.306000,0.193000,0.480000,0.211000,0.144000,0.160000
2018-06-30 20:00:00,6.816000,1.477000,2.462000,7.235000,2.514000,11.591000,1.596000,9.357000,0.451000,0.746000,...,0.227,0.889,0.102,0.557,0.284000,0.194000,0.459000,0.244000,0.193000,0.165000
2018-06-30 21:00:00,6.402000,1.432000,2.013000,5.145000,2.527000,10.766000,1.379000,15.048000,0.480000,0.455000,...,0.244,0.939,0.104,0.323,0.298000,0.259000,0.425000,0.915000,0.289000,0.146000


In [5]:
df1 = df*1000 # kWh -> Wh로 변환

ConfigProto 문제 해결


https://stackoverflow.com/questions/56127592/attributeerror-module-tensorflow-has-no-attribute-configproto


Session 문제 해결


https://stackoverflow.com/questions/55142951/tensorflow-2-0-attributeerror-module-tensorflow-has-no-attribute-session

In [7]:
list_submission = []
look_back = 24
data_dim = 1
which_data_frame = df1.columns
#  앞의 숫자는 Unit 개수
# input_shape = (data_size, time_steps, features)  그러나 모델을 돌릴 땐, data_size는 따로 넘기지 않음.
# batch_input_sshape = (data_size, time_steps, features) 주의점 data_size % batch_size == 0

# look_back = timesteip = seq_length
# data_dim ( 들어오는 데이터 열 개수)
# output_dim = 24 (hour: 나가게 하려는 값)

# convert nparray
for k in range(len(which_data_frame)):
    globals()['y_pred_list{}'.format(k)] = []
    nparr = df1.iloc[:, k].to_frame()
    nparr.astype('float32')
    print(nparr)

    # normalization
    scaler = MinMaxScaler(feature_range=(0, 1))
    nptf = scaler.fit_transform(nparr)
    
    # split train, test
    train_size = int(len(nptf) * 0.7)
    test_size = len(nptf) - train_size
    train, test = nptf[0:train_size], nptf[train_size:len(nptf)]
    print(len(train), len(test))

    
    def create_dataset(dataset, look_back):
        dataX, dataY = [], []
        for i in range(len(dataset)-look_back-1):
            a = dataset[i:(i + look_back)]
            dataX.append(a)
            dataY.append(dataset[i + look_back])
        return np.array(dataX), np.array(dataY)

    # create dataset for learning
    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)

    # reshape input to be [samples, time steps, features]
    trainX = np.reshape(trainX, (-1, look_back, data_dim))
    testX = np.reshape(testX, (-1, look_back, data_dim))


    # simple lstm network learning
    model = Sequential()
    model.add(LSTM(4, input_shape=(look_back, data_dim))) 
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics = ['accuracy'])
    es = EarlyStopping(monitor = 'val_loss', patience = 15)
    model.fit(trainX, trainY, validation_split = 0.3, epochs=100, batch_size=1, verbose=2, callbacks = [es])


    # make prediction
    testPredict = model.predict(testX)
    testPredict = scaler.inverse_transform(testPredict)
    testY = scaler.inverse_transform(testY)
    testScore = math.sqrt(mean_squared_error(testY, testPredict))
    print('Train Score: %.2f RMSE' % testScore)

    # predict last value (or tomorrow?)
    lastX = nptf[-24:]
    lastX = np.reshape(lastX, (-1, look_back, data_dim))
    lastY_pred = model.predict(lastX)
    lastY = scaler.inverse_transform(lastY_pred)
    print('Predict the value of final day: %d' % lastY)  
    lastY_pred

    # predict last value (or tomorrow?)
    number = 1
    lastY_pred = 0
    for i in range(look_back):
        lastX = nptf[-look_back: ]
        lastX = np.reshape(lastX, (-1, look_back, data_dim))
        lastY_pred = model.predict(lastX)
        lastY = (scaler.inverse_transform(lastY_pred))/1000
        nptf = np.concatenate((nptf, lastY_pred))

        print('Predict value of column number {}: {}'.format(k, lastY[0][0]))  # 데이터 입력 마지막 다음날 종가 예측
        globals()['y_pred_list{}'.format(k)].append(lastY[0][0])
    list_submission.append(globals()['y_pred_list{}'.format(k)])
        

                              x5
Time                            
2017-08-29 10:00:00  6373.729865
2017-08-29 11:00:00  6373.904713
2017-08-29 12:00:00  6374.295230
2017-08-29 13:00:00  6374.804149
2017-08-29 14:00:00  6375.378070
...                          ...
2018-06-30 19:00:00  6937.000000
2018-06-30 20:00:00  6816.000000
2018-06-30 21:00:00  6402.000000
2018-06-30 22:00:00  7184.000000
2018-06-30 23:00:00  7142.000000

[7334 rows x 1 columns]
5133 2201
Train on 3575 samples, validate on 1533 samples
Epoch 1/100
3575/3575 - 18s - loss: 0.0012 - accuracy: 0.0000e+00 - val_loss: 0.0010 - val_accuracy: 0.0000e+00
Epoch 2/100
3575/3575 - 15s - loss: 7.1399e-04 - accuracy: 0.0000e+00 - val_loss: 6.6442e-04 - val_accuracy: 0.0000e+00
Epoch 3/100
3575/3575 - 15s - loss: 6.7090e-04 - accuracy: 0.0000e+00 - val_loss: 5.8612e-04 - val_accuracy: 0.0000e+00
Epoch 4/100
3575/3575 - 15s - loss: 6.5323e-04 - accuracy: 0.0000e+00 - val_loss: 5.8174e-04 - val_accuracy: 0.0000e+00
Epoch 5/100
3575

Epoch 65/100
3575/3575 - 15s - loss: 1.2993e-05 - accuracy: 0.0000e+00 - val_loss: 5.7600e-06 - val_accuracy: 0.0000e+00
Epoch 66/100
3575/3575 - 15s - loss: 1.5734e-05 - accuracy: 0.0000e+00 - val_loss: 1.8580e-06 - val_accuracy: 0.0000e+00
Epoch 67/100
3575/3575 - 15s - loss: 9.2015e-06 - accuracy: 0.0000e+00 - val_loss: 1.8821e-05 - val_accuracy: 0.0000e+00
Epoch 68/100
3575/3575 - 15s - loss: 1.7473e-05 - accuracy: 0.0000e+00 - val_loss: 2.0581e-06 - val_accuracy: 0.0000e+00
Train Score: 633.77 RMSE
Predict the value of final day: 7154
Predict value of column number 0: 7.154442310333252
Predict value of column number 0: 7.033098220825195
Predict value of column number 0: 6.980120658874512
Predict value of column number 0: 7.015205383300781
Predict value of column number 0: 7.119657516479492
Predict value of column number 0: 7.233257293701172
Predict value of column number 0: 7.225847244262695
Predict value of column number 0: 7.06596040725708
Predict value of column number 0: 6.732

Train Score: 580.38 RMSE
Predict the value of final day: 1302
Predict value of column number 1: 1.3025649785995483
Predict value of column number 1: 1.30219304561615
Predict value of column number 1: 1.302061915397644
Predict value of column number 1: 1.3020079135894775
Predict value of column number 1: 1.3019847869873047
Predict value of column number 1: 1.3019742965698242
Predict value of column number 1: 1.3019696474075317
Predict value of column number 1: 1.301967740058899
Predict value of column number 1: 1.3019658327102661
Predict value of column number 1: 1.301963210105896
Predict value of column number 1: 1.3019616603851318
Predict value of column number 1: 1.3019592761993408
Predict value of column number 1: 1.3019540309906006
Predict value of column number 1: 1.3019466400146484
Predict value of column number 1: 1.3019343614578247
Predict value of column number 1: 1.301918387413025
Predict value of column number 1: 1.301893949508667
Predict value of column number 1: 1.30186951

Train Score: 1311.31 RMSE
Predict the value of final day: 2217
Predict value of column number 2: 2.2179980278015137
Predict value of column number 2: 2.052260160446167
Predict value of column number 2: 2.078962802886963
Predict value of column number 2: 2.0911591053009033
Predict value of column number 2: 2.0723564624786377
Predict value of column number 2: 2.0691189765930176
Predict value of column number 2: 2.1135716438293457
Predict value of column number 2: 2.214963674545288
Predict value of column number 2: 2.3445169925689697
Predict value of column number 2: 2.485725164413452
Predict value of column number 2: 2.6164486408233643
Predict value of column number 2: 2.72908616065979
Predict value of column number 2: 2.8330070972442627
Predict value of column number 2: 2.914578437805176
Predict value of column number 2: 2.976187229156494
Predict value of column number 2: 3.0206186771392822
Predict value of column number 2: 3.050494432449341
Predict value of column number 2: 3.070372581

Train on 3575 samples, validate on 1533 samples
Epoch 1/100
3575/3575 - 17s - loss: 3.1086e-04 - accuracy: 0.0000e+00 - val_loss: 6.9722e-05 - val_accuracy: 0.0000e+00
Epoch 2/100
3575/3575 - 15s - loss: 7.1129e-05 - accuracy: 0.0000e+00 - val_loss: 7.6049e-05 - val_accuracy: 0.0000e+00
Epoch 3/100
3575/3575 - 15s - loss: 6.4535e-05 - accuracy: 0.0000e+00 - val_loss: 5.2400e-05 - val_accuracy: 0.0000e+00
Epoch 4/100
3575/3575 - 15s - loss: 4.8496e-05 - accuracy: 0.0000e+00 - val_loss: 3.4736e-05 - val_accuracy: 0.0000e+00
Epoch 5/100
3575/3575 - 15s - loss: 4.0548e-05 - accuracy: 0.0000e+00 - val_loss: 3.4784e-05 - val_accuracy: 0.0000e+00
Epoch 6/100
3575/3575 - 15s - loss: 3.5924e-05 - accuracy: 0.0000e+00 - val_loss: 2.9760e-05 - val_accuracy: 0.0000e+00
Epoch 7/100
3575/3575 - 15s - loss: 3.2641e-05 - accuracy: 0.0000e+00 - val_loss: 2.4650e-05 - val_accuracy: 0.0000e+00
Epoch 8/100
3575/3575 - 15s - loss: 2.6190e-05 - accuracy: 0.0000e+00 - val_loss: 1.7459e-05 - val_accuracy: 0.0

...

Train on 3575 samples, validate on 1533 samples
Epoch 1/100
3575/3575 - 17s - loss: 0.0058 - accuracy: 2.7972e-04 - val_loss: 0.0054 - val_accuracy: 0.0000e+00
Epoch 2/100
3575/3575 - 15s - loss: 0.0052 - accuracy: 2.7972e-04 - val_loss: 0.0054 - val_accuracy: 0.0000e+00
Epoch 3/100
3575/3575 - 15s - loss: 0.0052 - accuracy: 2.7972e-04 - val_loss: 0.0052 - val_accuracy: 0.0000e+00
Epoch 4/100
3575/3575 - 15s - loss: 0.0051 - accuracy: 2.7972e-04 - val_loss: 0.0051 - val_accuracy: 0.0000e+00
Epoch 5/100
3575/3575 - 15s - loss: 0.0051 - accuracy: 2.7972e-04 - val_loss: 0.0051 - val_accuracy: 0.0000e+00
Epoch 6/100
3575/3575 - 15s - loss: 0.0051 - accuracy: 2.7972e-04 - val_loss: 0.0051 - val_accuracy: 0.0000e+00
Epoch 7/100
3575/3575 - 15s - loss: 0.0051 - accuracy: 2.7972e-04 - val_loss: 0.0051 - val_accuracy: 0.0000e+00
Epoch 8/100
3575/3575 - 15s - loss: 0.0051 - accuracy: 2.7972e-04 - val_loss: 0.0051 - val_accuracy: 0.0000e+00
Epoch 9/100
3575/3575 - 15s - loss: 0.0051 - accuracy: 2

Epoch 74/100
3575/3575 - 15s - loss: 0.0044 - accuracy: 2.7972e-04 - val_loss: 0.0050 - val_accuracy: 0.0000e+00
Epoch 75/100
3575/3575 - 15s - loss: 0.0043 - accuracy: 2.7972e-04 - val_loss: 0.0048 - val_accuracy: 0.0000e+00
Epoch 76/100
3575/3575 - 15s - loss: 0.0043 - accuracy: 2.7972e-04 - val_loss: 0.0050 - val_accuracy: 0.0000e+00
Epoch 77/100
3575/3575 - 15s - loss: 0.0043 - accuracy: 2.7972e-04 - val_loss: 0.0048 - val_accuracy: 0.0000e+00
Epoch 78/100
3575/3575 - 15s - loss: 0.0043 - accuracy: 2.7972e-04 - val_loss: 0.0049 - val_accuracy: 0.0000e+00
Epoch 79/100
3575/3575 - 15s - loss: 0.0043 - accuracy: 2.7972e-04 - val_loss: 0.0048 - val_accuracy: 0.0000e+00
Epoch 80/100
3575/3575 - 15s - loss: 0.0043 - accuracy: 2.7972e-04 - val_loss: 0.0049 - val_accuracy: 0.0000e+00
Epoch 81/100
3575/3575 - 15s - loss: 0.0042 - accuracy: 2.7972e-04 - val_loss: 0.0051 - val_accuracy: 0.0000e+00
Epoch 82/100
3575/3575 - 15s - loss: 0.0043 - accuracy: 2.7972e-04 - val_loss: 0.0049 - val_accu

Epoch 47/100
3575/3575 - 15s - loss: 0.0035 - accuracy: 2.7972e-04 - val_loss: 0.0016 - val_accuracy: 0.0000e+00
Epoch 48/100
3575/3575 - 15s - loss: 0.0035 - accuracy: 2.7972e-04 - val_loss: 0.0016 - val_accuracy: 0.0000e+00
Epoch 49/100
3575/3575 - 15s - loss: 0.0035 - accuracy: 2.7972e-04 - val_loss: 0.0016 - val_accuracy: 0.0000e+00
Epoch 50/100
3575/3575 - 15s - loss: 0.0035 - accuracy: 2.7972e-04 - val_loss: 0.0017 - val_accuracy: 0.0000e+00
Epoch 51/100
3575/3575 - 15s - loss: 0.0035 - accuracy: 2.7972e-04 - val_loss: 0.0017 - val_accuracy: 0.0000e+00
Epoch 52/100
3575/3575 - 15s - loss: 0.0035 - accuracy: 2.7972e-04 - val_loss: 0.0016 - val_accuracy: 0.0000e+00
Epoch 53/100
3575/3575 - 15s - loss: 0.0035 - accuracy: 2.7972e-04 - val_loss: 0.0016 - val_accuracy: 0.0000e+00
Epoch 54/100
3575/3575 - 15s - loss: 0.0035 - accuracy: 2.7972e-04 - val_loss: 0.0016 - val_accuracy: 0.0000e+00
Epoch 55/100
3575/3575 - 15s - loss: 0.0035 - accuracy: 2.7972e-04 - val_loss: 0.0016 - val_accu

Epoch 3/100
3575/3575 - 15s - loss: 0.0041 - accuracy: 0.0000e+00 - val_loss: 0.0057 - val_accuracy: 0.0000e+00
Epoch 4/100
3575/3575 - 15s - loss: 0.0041 - accuracy: 0.0000e+00 - val_loss: 0.0056 - val_accuracy: 0.0000e+00
Epoch 5/100
3575/3575 - 15s - loss: 0.0041 - accuracy: 0.0000e+00 - val_loss: 0.0056 - val_accuracy: 0.0000e+00
Epoch 6/100
3575/3575 - 15s - loss: 0.0041 - accuracy: 0.0000e+00 - val_loss: 0.0056 - val_accuracy: 0.0000e+00
Epoch 7/100
3575/3575 - 15s - loss: 0.0040 - accuracy: 0.0000e+00 - val_loss: 0.0056 - val_accuracy: 0.0000e+00
Epoch 8/100
3575/3575 - 15s - loss: 0.0040 - accuracy: 0.0000e+00 - val_loss: 0.0057 - val_accuracy: 0.0000e+00
Epoch 9/100
3575/3575 - 15s - loss: 0.0040 - accuracy: 0.0000e+00 - val_loss: 0.0056 - val_accuracy: 0.0000e+00
Epoch 10/100
3575/3575 - 15s - loss: 0.0040 - accuracy: 0.0000e+00 - val_loss: 0.0055 - val_accuracy: 0.0000e+00
Epoch 11/100
3575/3575 - 15s - loss: 0.0040 - accuracy: 0.0000e+00 - val_loss: 0.0056 - val_accuracy: 0

Epoch 7/100
3575/3575 - 15s - loss: 0.0024 - accuracy: 0.0000e+00 - val_loss: 0.0111 - val_accuracy: 0.0000e+00
Epoch 8/100
3575/3575 - 15s - loss: 0.0023 - accuracy: 0.0000e+00 - val_loss: 0.0113 - val_accuracy: 0.0000e+00
Epoch 9/100
3575/3575 - 15s - loss: 0.0023 - accuracy: 0.0000e+00 - val_loss: 0.0114 - val_accuracy: 0.0000e+00
Epoch 10/100
3575/3575 - 15s - loss: 0.0024 - accuracy: 0.0000e+00 - val_loss: 0.0114 - val_accuracy: 0.0000e+00
Epoch 11/100
3575/3575 - 15s - loss: 0.0023 - accuracy: 0.0000e+00 - val_loss: 0.0113 - val_accuracy: 0.0000e+00
Epoch 12/100
3575/3575 - 15s - loss: 0.0023 - accuracy: 0.0000e+00 - val_loss: 0.0112 - val_accuracy: 0.0000e+00
Epoch 13/100
3575/3575 - 15s - loss: 0.0023 - accuracy: 0.0000e+00 - val_loss: 0.0114 - val_accuracy: 0.0000e+00
Epoch 14/100
3575/3575 - 15s - loss: 0.0023 - accuracy: 0.0000e+00 - val_loss: 0.0113 - val_accuracy: 0.0000e+00
Epoch 15/100
3575/3575 - 15s - loss: 0.0023 - accuracy: 0.0000e+00 - val_loss: 0.0116 - val_accurac

In [10]:
submission=pd.DataFrame(list_submission).T
submission

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,7.154442,1.302565,2.217998,3.100928,2.748381,3.129092,1.176652,7.841544,0.169238,0.309152,...,0.408453,0.258295,0.115826,0.269708,0.256253,0.205277,0.359713,0.355406,0.234185,0.257280
1,7.033098,1.302193,2.052260,2.947774,2.748613,1.946161,1.273577,7.809060,0.136489,0.402257,...,0.425591,0.280589,0.133467,0.245909,0.248297,0.225379,0.327057,0.294283,0.217995,0.271485
2,6.980121,1.302062,2.078963,2.838590,2.748855,0.783212,1.392146,7.810524,0.126569,0.437114,...,0.444397,0.278998,0.151007,0.229619,0.242692,0.235267,0.201781,0.257944,0.214012,0.273845
3,7.015205,1.302008,2.091159,2.761670,2.749094,0.049670,1.455651,7.746837,0.198021,0.501322,...,0.465389,0.274792,0.177308,0.218489,0.239747,0.241177,0.295005,0.239744,0.212428,0.268415
4,7.119658,1.301985,2.072356,2.706880,2.749302,-0.345362,1.498339,7.620488,0.277160,0.524371,...,0.486158,0.270720,0.200562,0.210779,0.238096,0.245230,0.329820,0.233854,0.211632,0.259263
5,7.233257,1.301974,2.069119,2.667567,2.749487,-0.768637,1.524407,7.390101,0.336650,0.526003,...,0.506295,0.270259,0.216500,0.205346,0.237609,0.248132,0.351793,0.236286,0.211159,0.249764
6,7.225847,1.301970,2.113572,2.640254,2.749654,-1.474563,1.537739,7.038311,0.380108,0.520758,...,0.526293,0.278513,0.230965,0.201442,0.238236,0.250166,0.358593,0.245833,0.210865,0.242171
7,7.065960,1.301968,2.214964,2.619906,2.749903,-1.713208,1.546194,6.549625,0.411070,0.513548,...,0.546330,0.293118,0.193438,0.198579,0.239733,0.251501,0.405937,0.264298,0.210687,0.237651
8,6.732726,1.301966,2.344517,2.605208,2.750160,-1.064496,1.566580,5.184606,0.433203,0.506878,...,0.566515,0.298705,0.163648,0.196439,0.241823,0.252281,0.418261,0.290439,0.210582,0.236377
9,6.476751,1.301963,2.485725,2.594695,2.750380,-0.564235,1.608170,4.977358,0.449270,0.501705,...,0.587105,0.285203,0.145596,0.194809,0.244174,0.252643,0.414072,0.311830,0.210521,0.237735


In [12]:
submission.to_csv('submission_LSTM.csv', index=False)